In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt # Visualization
import seaborn as sns # Visualization



# Data Preparation

---

## Data Extraction

In [2]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")

In [3]:
sample_submission.shape

In [4]:
train.shape

In [5]:
train.info()

In [6]:
test.shape

In [7]:
test.info()

## Data Concatenation

In [8]:
data = pd.concat([train, test], sort = False)
data.shape

In [9]:
data.info()

In [10]:
data.head()

## Null Check

In [11]:
null_cols = [col for col in data.iloc[: , : -1].columns if data[col].isnull().sum() != 0]
null_cols

In [12]:
float_cols = [col for col in data.iloc[: , 1 : -1].columns if data[col].dtype == "float64"]
len(float_cols)

In [13]:
data.columns

In [14]:
CHUNMEIHONG = '#f1939c'
QIUBOLAN = '#8abcd1'

fig, axes = plt.subplots(20, 5, figsize = (16, 48))
axes = axes.flatten()

sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

for idx, ax in enumerate(axes):
    sns.kdeplot(
        data = train.loc[: , 'f0' :],
        ax = ax,
        hue = 'target',
        fill = True,
        x = f'f{idx}',
        palette = [f'{CHUNMEIHONG}', f'{QIUBOLAN}'],
        legend = idx == 0,
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.spines['left'].set_visible(False)
    ax.set_title(
        f'f{idx}',
        loc = 'right',
        weight = 'bold',
        fontsize = 10,
    )

fig.supxlabel('Probability Density Function Estimation', ha = 'center', fontweight = 'bold')
fig.tight_layout()

In [21]:
CHUNMEIHONG = '#f1939c'
QIUBOLAN = '#8abcd1'

fig, axes = plt.subplots(2, 5, figsize = (16, 4.8))
axes = axes.flatten()

sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

for idx, ax in enumerate(axes):
    sns.kdeplot(
        data = train.iloc[: , [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1]],
        ax = ax,
        hue = 'target',
        fill = True,
        x = f'f{idx}',
        palette = [f'{CHUNMEIHONG}', f'{QIUBOLAN}'],
        legend = idx == 0,
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.spines['left'].set_visible(False)
    ax.set_title(
        f'f{idx}',
        loc = 'right',
        weight = 'bold',
        fontsize = 10,
    )

fig.supxlabel('Probability Density Function Estimation', ha = 'center', fontweight = 'bold')
fig.tight_layout()

In [16]:
train.iloc[: , [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1]]

In [17]:
sns.color_palette("light:#0ABAB5", n_colors = 5, desat = .9)

In [18]:
palettes = []

for i in np.linspace(1, 0.05, num = 5):
    for j in range(1):
        palette = []
        palette.append(sns.color_palette("light:#F7CAC9", n_colors = 5, desat = i)[j])
        palette.append(sns.color_palette("light:#92A8D1", n_colors = 5, desat = i)[j])
        palettes.append(palette)
        
palettes

In [19]:
train.loc[: , 'f0' :].columns